<a href="https://colab.research.google.com/github/cagBRT/SentimentTextAnalysis/blob/master/Text_CNN_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the entire repo.
%cd /content/
!git clone  https://github.com/cagBRT/SentimentTextAnalysis.git cloned-repo
%cd cloned-repo
!ls

/content
Cloning into 'cloned-repo'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 176 (delta 5), reused 0 (delta 0), pack-reused 163
Receiving objects: 100% (176/176), 1.76 MiB | 3.60 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/cloned-repo
amazon_cells_labelled.txt  imdb_labelled.txt  Sentiment_Text_Analysis.ipynb
coffeedata.csv		   README.md	      Text_CNN_interpretation.ipynb
images			   readme.txt	      yelp_labelled.txt


## CNN intepretation for text classification
Creating a heatmap identify words and phrases which correlate to model predictions

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
from keras.models import Model
from keras import backend as K 
from keras.models import Sequential
from keras import layers
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer


pd.options.display.max_rows
pd.set_option('display.max_colwidth', -1)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [3]:
df = pd.read_csv('/content/cloned-repo/coffeedata.csv') #from https://drive.google.com/open?id=17jYWjTHrwMegjdTYs0YLXyc9PDRLOSi1
#data clean:
df['light_dark'] = np.where(df.roast=='Very Dark','Dark',df.roast)
df = df.loc[(df.light_dark=='Dark')|(df.light_dark=='Light')]
df = df.loc[(df.blind_assesment.str.len()>10)]
df['y'] = np.where(df.light_dark=='Light',1,0)
df.light_dark.value_counts()
df.columns

Index(['rating', 'company', 'coffee_name', 'location', 'origin', 'roast',
       'review_date', 'agtron', 'acidity', 'body', 'flavor', 'blind_assesment',
       'notes', 'bottom_line', 'light_dark', 'y'],
      dtype='object')

In [4]:
X = df.blind_assesment.values
y = df.y.values

tokenizer = Tokenizer(num_words=4000)
tokenizer.fit_on_texts(X)

X = tokenizer.texts_to_sequences(X)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

maxlen = 200
embedding_dim = 50

X = pad_sequences(X, padding='post', maxlen=maxlen)


sequence_input = layers.Input(shape=(maxlen,), dtype='int32')
embedded_sequences = layers.Embedding(vocab_size, embedding_dim, input_length=maxlen)(sequence_input)
l_cov1  = layers.Conv1D(317, 3, activation='relu')(embedded_sequences)
l_pool1 = layers.MaxPooling1D(2)(l_cov1)
l_cov2  = layers.Conv1D(317, 1, activation='relu')(l_pool1)
l_cov3  = layers.Conv1D(317, 2, activation='relu')(l_cov2)
l_pool3 = layers.GlobalMaxPooling1D()(l_cov3)  # global max pooling
l_bnorm = layers.BatchNormalization()(l_pool3)
l_dense = layers.Dense(128, activation='relu')(l_pool3)
preds   = layers.Dense(1, activation='sigmoid',name='preds')(l_dense)


model = Model(sequence_input, outputs=preds)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

model.fit(X, y, epochs=3, validation_split=0.1, batch_size=10)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 50)           81550     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 317)          47867     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 99, 317)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 99, 317)           100806    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 98, 317)           201295    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 317)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 481 samples, validate on 54 samples
Epoch 1/3
481/481 [==============================] - 7s 14ms/step - loss: 0.5196 - accuracy: 0.7859 - val_loss: 1.1151 - val_accuracy: 0.1111
Epoch 2/3
481/481 [==============================] - 0s 499us/step - loss: 0.3594 - accuracy: 0.8108 - val_loss: 1.1105 - val_accuracy: 0.3704
Epoch 3/3
481/481 [==============================] - 0s 480us/step - loss: 0.1789 - accuracy: 0.9397 - val_loss: 3.1192 - val_accuracy: 0.3148


In [6]:
#@title ## Coffee darkness predictor
#@markdown Try-it-out by pasting or typing your coffee taste description below:
type_here = "A blend of beans spanning the globe. Roasted with a flavor as intense as the flame that birthed it, as rich as the soil it was grown in, and as smooth as the law will allow."#@param {type:"string"}
maxlen = 200
Xtst = tokenizer.texts_to_sequences([type_here])
Xtst = pad_sequences(Xtst, padding='post', maxlen=maxlen)

y_pred = model.predict(Xtst)

class_idx = np.argmax(y_pred[0]) #not needed in this case as only two classes
class_output = model.output[:, class_idx]
last_conv_layer = model.get_layer("conv1d_3")

grads = K.gradients(class_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads)
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([Xtst])

    
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap,0)
heatmap /= np.max(heatmap)#normalise values in the prediction


norm_len = maxlen/last_conv_layer.output_shape[1] # find the ratio of the text vs the conv layer length

html = ""
if y_pred[0]>0.5:
  pred = 'light'
else:
  pred = 'dark'
html += "<span><h3>Based on the description, the model believes that this is a {} coffee roast. ".format(pred)
html += "<small><br>Confidence: {:.0f}%<br><br></small></h3></span>".format(abs(((y_pred[0][0]*100)-50)*2))
for j,i in enumerate(tokenizer.sequences_to_texts(Xtst)[0].split()):
  html += "<span style='background-color:rgba({},0,150,{})'>{} </span>".format(heatmap[math.floor(j/norm_len)]*255,heatmap[math.floor(j/norm_len)]-0.3,i)

HTML(html)
